In [110]:
import pandas as pd
import numpy as np
from datetime import datetime

In [111]:
# read raw data and import into df with pandas
df = pd.read_csv('C:/Users/Cameron/Desktop/CS5530/Assignment2 - CS5530/train.csv')
print(df.shape)

#check df for null/missing values and return summary
missing_values = df.isnull().sum()
print(missing_values)

(5847, 14)
Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [112]:
# drop where missing values for mileage, engine, power
df = df.dropna(subset=['Mileage', 'Engine', 'Power'])

# replace missing values for seats with the mode
seats_mode = df['Seats'].mode()[0]
df['Seats'].fillna(seats_mode, inplace=True)

# store rows that have both price and new price values
rows_with_both_prices = df.dropna(subset=['Price', 'New_Price'])

# clone rows and modify with values that should be found in a new car (i.e, km driven = 0, price = same as new price, etc)
new_price_rows = rows_with_both_prices.copy()
new_price_rows['Kilometers_Driven'] = 0 
new_price_rows['Price'] = new_price_rows['New_Price']  
new_price_rows['Owner_Type'] = 'First' 

# remove original set of rows that were cloned
df.drop(rows_with_both_prices.index, inplace=True)

# recombine df and drop new_price column
final_df = pd.concat([df, new_price_rows]).sort_index()
final_df.reset_index(drop=True, inplace=True)
final_df = df.drop('New_Price', axis=1)

# new df shape
final_shape = final_df.shape
first_few_rows = final_df.head()

In [113]:
# standardizing the units and removing
final_df['Mileage'] = final_df['Mileage'].str.extract('(\d+\.\d+|\d+)').astype(float)
final_df['Engine'] = final_df['Engine'].str.extract('(\d+)').astype(float)
final_df['Power'] = final_df['Power'].str.replace('null bhp', '0 bhp') 
final_df['Power'] = final_df['Power'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [115]:
# One-hot encode
final_df = pd.get_dummies(final_df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# car age
current_year = datetime.now().year
final_df['Car_Age'] = current_year - final_df['Year']


In [105]:
# save df to CSV
final_df.to_csv('C:/Users/Cameron/Desktop/CS5530/Assignment2 - CS5530/modified_train.csv', index=False)

In [118]:
# select operation
selected_df = final_df[['Price', 'Car_Age']]

# filter operation
filtered_df = final_df[final_df['Car_Age'] == 1]

# rename operation
renamed_df = final_df.rename(columns={'Car_Age': 'Vehicle Age'})

# mutate operation
final_df['Price_per_Year'] = final_df['Price'] / final_df['Car_Age']

# arrange operation
sorted_df = final_df.sort_values('Price', ascending=False)

# summarize operation
average_price_by_location = final_df.groupby('Location')['Price'].mean().reset_index()